In [ ]:
import pandas as pd
import numpy as np

# import matplotlib.pyplot as plt
import yaml
import os
import glob
import sys
sys.path.append('../')
from utils import *

BASE_URL = 'C:/Users/Admin/Desktop/DATN/DATA/SVIC-APT-2021/'


In [ ]:
# file_paths = [f"{BASE_URL}phase{i}.csv" for i in range(1, 5)]

# df_temp = [pd.read_csv(file_path) for file_path in file_paths]
# df = pd.concat(df_temp, ignore_index=True)

In [ ]:
# df = df.dropna(axis=1, how='all')
df = pd.read_csv(BASE_URL+'Training.csv')
df2 = pd.read_csv(BASE_URL+'Testing.csv')

In [ ]:
df.value_counts('Label')

In [ ]:
df2.value_counts('Label')

In [ ]:
list_features = ['Flow Duration', 'Total Fwd Packet', 'Total Bwd packets',
       'Total Length of Fwd Packet', 'Total Length of Bwd Packet',
       'Fwd Packet Length Max', 'Fwd Packet Length Min',
       'Fwd Packet Length Mean', 'Fwd Packet Length Std',
       'Bwd Packet Length Max', 'Bwd Packet Length Min',
       'Bwd Packet Length Mean', 'Bwd Packet Length Std', 'Flow Bytes/s',
       'Flow Packets/s', 'Flow IAT Mean', 'Flow IAT Std', 'Flow IAT Max',
       'Flow IAT Min', 'Fwd IAT Total', 'Fwd IAT Mean', 'Fwd IAT Std',
       'Fwd IAT Max', 'Fwd IAT Min', 'Bwd IAT Total', 'Bwd IAT Mean',
       'Bwd IAT Std', 'Bwd IAT Max', 'Bwd IAT Min', 'Fwd PSH Flags',
       'Bwd PSH Flags', 'Fwd URG Flags', 'Bwd URG Flags', 'Fwd Header Length',
       'Bwd Header Length', 'Fwd Packets/s', 'Bwd Packets/s',
       'Packet Length Min', 'Packet Length Max', 'Packet Length Mean',
       'Packet Length Std', 'Packet Length Variance', 'FIN Flag Count',
       'SYN Flag Count', 'RST Flag Count', 'PSH Flag Count', 'ACK Flag Count',
       'URG Flag Count', 'CWR Flag Count', 'ECE Flag Count', 'Down/Up Ratio',
       'Average Packet Size', 'Fwd Segment Size Avg', 'Bwd Segment Size Avg',
       'Fwd Bytes/Bulk Avg', 'Fwd Packet/Bulk Avg', 'Fwd Bulk Rate Avg',
       'Bwd Bytes/Bulk Avg', 'Bwd Packet/Bulk Avg', 'Bwd Bulk Rate Avg',
       'Subflow Fwd Packets', 'Subflow Fwd Bytes', 'Subflow Bwd Packets',
       'Subflow Bwd Bytes', 'FWD Init Win Bytes', 'Bwd Init Win Bytes',
       'Fwd Act Data Pkts', 'Fwd Seg Size Min', 'Active Mean', 'Active Std',
       'Active Max', 'Active Min', 'Idle Mean', 'Idle Std', 'Idle Max',
       'Idle Min', 'Label']

df = df[list_features]
df2 = df2[list_features]


In [ ]:
cols_unique_val = []
cols_unique_val2 = []

# Dropping features contain unique value for every instance
for i in range(1,2):
    for col in df.columns:
        if df[col].nunique() == i:
            cols_unique_val.append(col)

for i in range(1,2):
    for col in df.columns:
        if df2[col].nunique() == i:
            cols_unique_val2.append(col)

print(f'Features contain unique value for every instance: {cols_unique_val}')
print(f'Features contain unique value for every instance: {cols_unique_val2}')

print(len(cols_unique_val))
print(len(cols_unique_val2))

df.drop(cols_unique_val, axis=1, inplace=True)
df2.drop(cols_unique_val, axis=1, inplace=True)


In [ ]:
df2

In [ ]:
df2.value_counts('Label')

In [ ]:
stage_mapping = {
    'NormalTraffic': 0,        
    'Reconnaissance': 1,     
    'InitialCompromise': 1,
    'LateralMovement': 2,
    'Pivoting': 2,
    'DataExfiltration': 3
}

df['Label'] = df['Label'].map(stage_mapping)
df2['Label'] = df2['Label'].map(stage_mapping)

In [ ]:
df.value_counts('Label')

In [ ]:
# df.value_counts('Stage')
df.replace([-np.inf, np.inf], np.nan, inplace=True)
df.dropna(axis=0, inplace=True)

# Dropping duplicate samples
df.drop_duplicates(inplace=True)

# df.value_counts('Stage')
df2.replace([-np.inf, np.inf], np.nan, inplace=True)
df2.dropna(axis=0, inplace=True)

# Dropping duplicate samples
df2.drop_duplicates(inplace=True)

In [ ]:
df.to_csv(BASE_URL + 'scvic_training.csv')
df2.to_csv(BASE_URL + 'scvic_test.csv')

-----------------------
-----------------------
-----------------------


In [117]:
df = pd.read_csv(BASE_URL + 'scvic_training.csv')
df2 = pd.read_csv(BASE_URL + 'scvic_test.csv')

<h2> Training </h2>


In [118]:
df.drop('Unnamed: 0', axis=1, inplace=True)
df2.drop('Unnamed: 0', axis=1, inplace=True)

In [119]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import MultinomialNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import LinearSVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix, classification_report
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import StandardScaler, OneHotEncoder

import xgboost as xgb


from imblearn.under_sampling import RandomUnderSampler
from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import TomekLinks
from imblearn.combine import SMOTETomek
from imblearn.pipeline import Pipeline, make_pipeline

In [130]:
encoder = OneHotEncoder(sparse_output=False, handle_unknown='ignore', drop='first')

X_train = df.drop('Label', axis=1)  # Features
Y_train = df['Label']  # Target variable

# Y_train = Y_train.values.reshape(-1, 1)
# Y_train = encoder.fit_transform(Y_train)

x_train, x_test, y_train, y_test = train_test_split(X_train, Y_train, stratify=Y_train, test_size=0.2, random_state=42)

X_test = df2.iloc[:, :-1].values
Y_test = df2.iloc[:, -1].values


In [ ]:
# scaler = StandardScaler()

# x_train = scaler.fit_transform(x_train)
# x_train = scaler.fit_transform(x_train)

# x_test = scaler.transform(x_test)


In [ ]:
rus = RandomUnderSampler(random_state=42)
x_train, y_train = rus.fit_resample(x_train, y_train)

RANDOM FOREST

In [131]:
rf_scvic = RandomForestClassifier(
    n_estimators=100,
    n_jobs=12,
)

In [132]:
rf_scvic.fit(x_train, y_train)

# from sklearn.model_selection import KFold, cross_val_score

# kf = KFold(n_splits=5, shuffle=True, random_state=42)

# # Perform cross-validation
# scores = cross_val_score(rf_scvic, x_train, y_train, cv=kf, scoring='accuracy')

# print("Cross-Validation Scores:", scores)
# print("Mean Accuracy:", scores.mean())
# print("Standard Deviation:", scores.std())

RandomForestClassifier(n_jobs=12)

In [133]:
y_pred = rf_scvic.predict(x_test)
# y_pred = encoder.inverse_transform(y_pred)
print(classification_report(y_test, y_pred))
# print(confusion_matrix(y_test, y_pred))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00     31174
           1       0.89      0.93      0.91       181
           2       0.96      0.98      0.97       570
           3       0.95      0.81      0.87       106

    accuracy                           1.00     32031
   macro avg       0.95      0.93      0.94     32031
weighted avg       1.00      1.00      1.00     32031



In [135]:
y_pred = rf_scvic.predict(X_test)
# y_pred = encoder.inverse_transform(y_pred)

print(classification_report(Y_test, y_pred))
print(confusion_matrix(Y_test, y_pred))

c:\Users\Admin\Desktop\DATN\RiDX_DATA_PREPROCESSING\venv\Lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(


              precision    recall  f1-score   support

           0       1.00      1.00      1.00     46168
           1       0.78      0.73      0.76       328
           2       0.78      0.93      0.85       501
           3       0.69      0.12      0.21        74

    accuracy                           1.00     47071
   macro avg       0.81      0.70      0.70     47071
weighted avg       1.00      1.00      0.99     47071

[[46135    11    22     0]
 [    6   239    80     3]
 [   10    23   467     1]
 [    1    32    32     9]]


LOGISTIC REGRESSION

In [ ]:
log_scvic = LogisticRegression(
    C=0.1,
    solver='sag',
    max_iter=200,
    multi_class='multinomial'
)

In [ ]:
log_scvic.fit(x_train, y_train)

In [ ]:
y_pred = log_scvic.predict(x_test)

# Calculate accuracy
print(classification_report(y_test, y_pred))
print(confusion_matrix(y_test, y_pred))



XGBOOST

In [136]:

xgb_scvic = xgb.XGBClassifier(objective='multi:softmax')

In [137]:
xgb_scvic.fit(x_train, y_train)

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=None, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=None, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=None, n_jobs=None,
              num_parallel_tree=None, objective='multi:softmax', ...)

In [138]:
y_pred = xgb_scvic.predict(x_test)

# Calculate accuracy
accuracy = accuracy_score(y_test, y_pred)
print(accuracy)

0.9988448690331242


In [142]:
y_pred = xgb_scvic.predict(X_test)

print(classification_report(Y_test, y_pred))
print(confusion_matrix(Y_test, y_pred))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00     46168
           1       0.92      0.67      0.77       328
           2       0.71      0.98      0.82       501
           3       0.77      0.27      0.40        74

    accuracy                           1.00     47071
   macro avg       0.85      0.73      0.75     47071
weighted avg       1.00      1.00      1.00     47071

[[46116     4    48     0]
 [    2   219   102     5]
 [    0    11   489     1]
 [    0     4    50    20]]


PREDICT


In [ ]:
df_zz = df2[df2['Label'] == 3]
df_zz.iloc[0]
t1 = df_zz.iloc[0, :-1].values.reshape(1, -1)

In [ ]:
df_z2 = df2[df2['Label'] == 2]
df_z2.iloc[4:10]
t2 = df_z2.iloc[:, :-1].values
print(t2)

In [ ]:
pred = xgb_scvic.predict(t2)

print(pred)


In [ ]:
import joblib

joblib.dump(xgb_scvic, 'xgb_svic.joblib')


# joblib.dump(rf_scvic, 'random_forest_scvic.pkl')
